In [1]:
import numpy as np
import pandas as pd
from aequilibrae.paths.public_transport import HyperpathGenerating
from numba import jit

RS = 124  # random seed

In [2]:
def create_vertices(n):
    x = np.linspace(0, 1, n)
    y = np.linspace(0, 1, n)
    xv, yv = np.meshgrid(x, y, indexing="xy")
    vertices = pd.DataFrame()
    vertices["x"] = xv.ravel()
    vertices["y"] = yv.ravel()
    return vertices

n = 10
vertices = create_vertices(n)
vertices

,x,y
0,0.000000,0.0
1,0.111111,0.0
2,0.222222,0.0
3,0.333333,0.0
4,0.444444,0.0
...,...,...
95,0.555556,1.0
96,0.666667,1.0
97,0.777778,1.0
98,0.888889,1.0


In [3]:
@jit
def create_edges_numba(n):
    m = 2 * n * (n - 1)
    tail = np.zeros(m, dtype=np.uint32)
    head = np.zeros(m, dtype=np.uint32)
    k = 0
    for i in range(n - 1):
        for j in range(n):
            tail[k] = i + j * n
            head[k] = i + 1 + j * n
            k += 1
            tail[k] = j + i * n
            head[k] = j + (i + 1) * n
            k += 1
    return tail, head

def create_edges(n, seed=124):
    tail, head = create_edges_numba(n)
    edges = pd.DataFrame()
    edges["tail"] = tail
    edges["head"] = head
    m = len(edges)
    rng = np.random.default_rng(seed=seed)
    edges["trav_time"] = rng.uniform(0.0, 1.0, m)
    edges["delay_base"] = rng.uniform(0.0, 1.0, m)
    edges['var_1_c'] = rng.uniform(0.0, 1.0, m)
    return edges

In [4]:
edges = create_edges(n, seed=RS)
edges

,tail,head,trav_time,delay_base,var_1_c
0,0,1,0.785253,0.287917,0.748310
1,0,10,0.785859,0.970429,0.125761
2,10,11,0.969136,0.854512,0.149131
3,1,11,0.748060,0.319959,0.573429
4,20,21,0.655551,0.047771,0.733364
...,...,...,...,...,...
175,87,97,0.025590,0.642800,0.716213
176,88,89,0.677292,0.482416,0.759359
177,88,98,0.096483,0.138376,0.579458
178,98,99,0.484049,0.136400,0.653367


In [5]:
alpha = 10.0

delay_base = edges.delay_base.values
indices = np.where(delay_base == 0.0)
delay_base[indices] = 1.0 # use this to prevent an error?
freq_base = 1.0 / delay_base
freq_base[indices] = np.inf
edges["freq_base"] = freq_base

if alpha == 0.0:
    edges["freq"] = np.inf
else:
    edges["freq"] = edges.freq_base / alpha

In [6]:
# Spiess & Florian
sf = HyperpathGenerating(
    edges, tail="tail", head="head", trav_time="trav_time", freq="freq"
)
sf._edges

,tail,head,trav_time,freq,edge_idx
0,0,1,0.785253,0.347322,0
1,0,10,0.785859,0.103047,1
2,10,11,0.969136,0.117026,2
3,1,11,0.748060,0.312540,3
4,20,21,0.655551,2.093321,4
...,...,...,...,...,...
175,87,97,0.025590,0.155569,175
176,88,89,0.677292,0.207290,176
177,88,98,0.096483,0.722671,177
178,98,99,0.484049,0.733140,178


In [8]:
sf._trav_time.shape

(180,)

In [10]:
sf._skim_cols.shape

(180,)

In [7]:
dest = n * n - 1
sf.run(origin=0, destination=dest, volume=1.0)

: 

In [20]:
edges_df = sf._edges

for i in range(sf._indptr[dest], sf._indptr[dest+1]):
    print(i)
    edge_idx = sf._edge_idx[i]
    print(edge_idx)
    print(sf._edges.trav_time[edge_idx])
    display(edges_df[edges_df.edge_idx == edge_idx])
edges_df[edges_df['head'] == 99]

178
178
0.48404915578440333


,tail,head,trav_time,freq,edge_idx,volume
178,98,99,0.484049,0.73314,178,0.88162


179
179
0.7912608054550185


,tail,head,trav_time,freq,edge_idx,volume
179,89,99,0.791261,0.149404,179,0.11838


,tail,head,trav_time,freq,edge_idx,volume
178,98,99,0.484049,0.733140,178,0.88162
179,89,99,0.791261,0.149404,179,0.11838


In [21]:
edges_df[edges_df.edge_idx == 179]

,tail,head,trav_time,freq,edge_idx,volume
179,89,99,0.791261,0.149404,179,0.11838


In [22]:
sf._edges

,tail,head,trav_time,freq,edge_idx,volume
0,0,1,0.785253,0.347322,0,0.771194
1,0,10,0.785859,0.103047,1,0.228806
2,10,11,0.969136,0.117026,2,0.000000
3,1,11,0.748060,0.312540,3,0.771194
4,20,21,0.655551,2.093321,4,0.216982
...,...,...,...,...,...,...
175,87,97,0.025590,0.155569,175,0.000000
176,88,89,0.677292,0.207290,176,0.000000
177,88,98,0.096483,0.722671,177,0.240592
178,98,99,0.484049,0.733140,178,0.881620


In [23]:
sf.u_i_vec

array([52.09625924, 50.7362477 , 50.29012703, 52.41375548, 50.82758145,
       50.11924603, 48.24846199, 48.39241413, 45.92053915, 43.88370516,
       43.54334338, 46.78859487, 45.72524603, 43.6762203 , 43.16977658,
       43.76784005, 43.34089596, 44.08791865, 43.09076564, 41.84234352,
       39.93075556, 38.8407916 , 38.09956231, 36.68694389, 34.69013688,
       35.00111374, 35.2733203 , 36.01293616, 35.79890705, 38.39658043,
       38.26240408, 37.22094869, 32.99118395, 32.62201292, 29.4599568 ,
       26.0790963 , 29.324187  , 28.81208559, 27.06670372, 31.05404947,
       30.83352321, 33.22772334, 32.64916301, 31.18388494, 27.07531288,
       24.71102132, 25.811869  , 21.14912348, 19.87453072, 26.68571608,
       29.8690225 , 27.20246995, 26.85435153, 25.74804623, 23.16957413,
       20.45977989, 22.49318111, 19.93196034, 18.63722004, 15.88485328,
       41.1443306 , 30.15607543, 25.93814664, 22.81153912, 21.51407745,
       17.89385237, 17.52250986, 14.27434694, 14.15979309, 13.32

In [25]:
edges_df = sf._edges

In [26]:
edges_df.loc[edges_df['head'] == 99]

,tail,head,trav_time,freq,edge_idx,volume
178,98,99,0.484049,0.733140,178,0.88162
179,89,99,0.791261,0.149404,179,0.11838


In [27]:
sf.u_i_vec

array([52.09625924, 50.7362477 , 50.29012703, 52.41375548, 50.82758145,
       50.11924603, 48.24846199, 48.39241413, 45.92053915, 43.88370516,
       43.54334338, 46.78859487, 45.72524603, 43.6762203 , 43.16977658,
       43.76784005, 43.34089596, 44.08791865, 43.09076564, 41.84234352,
       39.93075556, 38.8407916 , 38.09956231, 36.68694389, 34.69013688,
       35.00111374, 35.2733203 , 36.01293616, 35.79890705, 38.39658043,
       38.26240408, 37.22094869, 32.99118395, 32.62201292, 29.4599568 ,
       26.0790963 , 29.324187  , 28.81208559, 27.06670372, 31.05404947,
       30.83352321, 33.22772334, 32.64916301, 31.18388494, 27.07531288,
       24.71102132, 25.811869  , 21.14912348, 19.87453072, 26.68571608,
       29.8690225 , 27.20246995, 26.85435153, 25.74804623, 23.16957413,
       20.45977989, 22.49318111, 19.93196034, 18.63722004, 15.88485328,
       41.1443306 , 30.15607543, 25.93814664, 22.81153912, 21.51407745,
       17.89385237, 17.52250986, 14.27434694, 14.15979309, 13.32